In [1]:
pip install pandas numpy statsmodels streamlit plotly


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 128.8 MB/s eta 0:00:00


In [2]:
import pandas as pd

data = {
    "Date": pd.date_range(start="2025-03-01", periods=30, freq="D"),
    "Energy_kWh": [
        210,220,230,240,235,245,260,
        225,215,230,240,250,265,280,
        300,310,340,360,380,400,420,
        450,470,500,520,540,560,580,
        600,620
    ]
}

df = pd.DataFrame(data)
df


,Date,Energy_kWh
0,2025-03-01,210
1,2025-03-02,220
2,2025-03-03,230
3,2025-03-04,240
4,2025-03-05,235
5,2025-03-06,245
6,2025-03-07,260
7,2025-03-08,225
8,2025-03-09,215
9,2025-03-10,230


In [3]:
df["Is_Exam"] = ((df["Date"] >= "2025-03-15") & (df["Date"] <= "2025-03-30")).astype(int)
df


,Date,Energy_kWh,Is_Exam
0,2025-03-01,210,0
1,2025-03-02,220,0
2,2025-03-03,230,0
3,2025-03-04,240,0
4,2025-03-05,235,0
5,2025-03-06,245,0
6,2025-03-07,260,0
7,2025-03-08,225,0
8,2025-03-09,215,0
9,2025-03-10,230,0


In [4]:
df.set_index("Date", inplace=True)
df


,Energy_kWh,Is_Exam
Date,,
2025-03-01,210,0
2025-03-02,220,0
2025-03-03,230,0
2025-03-04,240,0
2025-03-05,235,0
2025-03-06,245,0
2025-03-07,260,0
2025-03-08,225,0
2025-03-09,215,0


In [5]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

ts = df["Energy_kWh"]

model = ExponentialSmoothing(
    ts,
    trend="add",
    seasonal="add",
    seasonal_periods=7
)

fit = model.fit()
forecast = fit.forecast(7)   # next 7 days (semester end forecast)

forecast


/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


,0
2025-03-31,651.410733
2025-04-01,675.998908
2025-04-02,696.267028
2025-04-03,720.962358
2025-04-04,746.334929
2025-04-05,762.385056
2025-04-06,780.970012


In [6]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df.index,
    y=df["Energy_kWh"],
    mode="lines",
    name="Historical Usage"
))

fig.add_trace(go.Scatter(
    x=forecast.index,
    y=forecast,
    mode="lines+markers",
    name="Forecast"
))

fig.update_layout(
    title="Library Energy Consumption Forecast",
    xaxis_title="Date",
    yaxis_title="Energy (kWh)"
)

fig.show()


In [7]:
predicted_value = round(forecast.iloc[0], 2)

fig_gauge = go.Figure(go.Indicator(
    mode="gauge+number",
    value=predicted_value,
    title={'text': "Next Day Predicted Energy Usage (kWh)"},
    gauge={
        'axis': {'range': [0, 700]},
        'bar': {'color': "darkblue"},
        'steps': [
            {'range': [0, 300], 'color': "lightgreen"},
            {'range': [300, 500], 'color': "orange"},
            {'range': [500, 700], 'color': "red"}
        ]
    }
))

fig_gauge.show()


In [8]:
print(f"Predicted energy usage for the next day: {predicted_value} kWh")


Predicted energy usage for the next day: 651.41 kWh
